In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import re
import numpy as np
from functools import reduce
from keras import preprocessing
from keras import layers
from keras import models

In [3]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split(r'(\W+)', sent) if x.strip()]

In [4]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [5]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data
            if not max_length or len(flatten(story)) < max_length]
    return data

In [6]:
def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (preprocessing.sequence.pad_sequences(inputs, maxlen=story_maxlen),
            preprocessing.sequence.pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))

In [7]:
### QA1

In [8]:
challenge = '../datasets/tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
train_stories = get_stories(open(challenge.format('train')))
test_stories = get_stories(open(challenge.format('test')))

In [9]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-


In [10]:
print('Compiling...')

# placeholders
input_sequence = layers.Input((story_maxlen,))
question = layers.Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = models.Sequential()
input_encoder_m.add(layers.Embedding(input_dim=vocab_size,
                                     output_dim=64))
input_encoder_m.add(layers.Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = models.Sequential()
input_encoder_c.add(layers.Embedding(input_dim=vocab_size,
                                     output_dim=query_maxlen))
input_encoder_c.add(layers.Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = models.Sequential()
question_encoder.add(layers.Embedding(input_dim=vocab_size,
                                      output_dim=64))
                                      #input_length=query_maxlen))
question_encoder.add(layers.Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = layers.dot([input_encoded_m, question_encoded], axes=(2, 2))
match = layers.Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = layers.add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = layers.Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = layers.concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = layers.CuDNNLSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = layers.Dropout(0.3)(answer)
answer = layers.Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = layers.Activation('softmax')(answer)

# build the final model
model = models.Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Compiling...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             1408        input_1[0][0]                    
__________________________________________________________________________________________________
sequenti

In [11]:
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 4s 416us/step - loss: 1.9331 - acc: 0.1691 - val_loss: 1.7905 - val_acc: 0.2280
Epoch 2/120
10000/10000 [==============================] - 2s 169us/step - loss: 1.7411 - acc: 0.2296 - val_loss: 1.6919 - val_acc: 0.3010
Epoch 3/120
10000/10000 [==============================] - 2s 169us/step - loss: 1.6147 - acc: 0.3360 - val_loss: 1.5005 - val_acc: 0.4080
Epoch 4/120
10000/10000 [==============================] - 2s 173us/step - loss: 1.5277 - acc: 0.3859 - val_loss: 1.4880 - val_acc: 0.4150
Epoch 5/120
10000/10000 [==============================] - 2s 172us/step - loss: 1.4997 - acc: 0.4026 - val_loss: 1.4395 - val_acc: 0.4380
Epoch 6/120
10000/10000 [==============================] - 2s 170us/step - loss: 1.4602 - acc: 0.4209 - val_loss: 1.3989 - val_acc

Epoch 55/120
10000/10000 [==============================] - 2s 167us/step - loss: 0.3756 - acc: 0.8570 - val_loss: 0.3690 - val_acc: 0.8590
Epoch 56/120
10000/10000 [==============================] - 2s 168us/step - loss: 0.3702 - acc: 0.8618 - val_loss: 0.3626 - val_acc: 0.8660
Epoch 57/120
10000/10000 [==============================] - 2s 170us/step - loss: 0.3492 - acc: 0.8630 - val_loss: 0.3512 - val_acc: 0.8600
Epoch 58/120
10000/10000 [==============================] - 2s 171us/step - loss: 0.3434 - acc: 0.8688 - val_loss: 0.3444 - val_acc: 0.8670
Epoch 59/120
10000/10000 [==============================] - 2s 170us/step - loss: 0.3312 - acc: 0.8736 - val_loss: 0.3385 - val_acc: 0.8710
Epoch 60/120
10000/10000 [==============================] - 2s 169us/step - loss: 0.3174 - acc: 0.8817 - val_loss: 0.3263 - val_acc: 0.8730
Epoch 61/120
10000/10000 [==============================] - 2s 172us/step - loss: 0.3080 - acc: 0.8835 - val_loss: 0.3142 - val_acc: 0.8840
Epoch 62/120
10000/1

10000/10000 [==============================] - 2s 165us/step - loss: 0.0731 - acc: 0.9753 - val_loss: 0.1718 - val_acc: 0.9500
Epoch 114/120
10000/10000 [==============================] - 2s 166us/step - loss: 0.0759 - acc: 0.9743 - val_loss: 0.1731 - val_acc: 0.9470
Epoch 115/120
10000/10000 [==============================] - 2s 171us/step - loss: 0.0745 - acc: 0.9753 - val_loss: 0.1655 - val_acc: 0.9520
Epoch 116/120
10000/10000 [==============================] - 2s 167us/step - loss: 0.0703 - acc: 0.9772 - val_loss: 0.1823 - val_acc: 0.9480
Epoch 117/120
10000/10000 [==============================] - 2s 169us/step - loss: 0.0710 - acc: 0.9740 - val_loss: 0.1809 - val_acc: 0.9400
Epoch 118/120
10000/10000 [==============================] - 2s 167us/step - loss: 0.0649 - acc: 0.9779 - val_loss: 0.1935 - val_acc: 0.9420
Epoch 119/120
10000/10000 [==============================] - 2s 170us/step - loss: 0.0740 - acc: 0.9749 - val_loss: 0.1781 - val_acc: 0.9450
Epoch 120/120
10000/10000 [